In [11]:
import joblib
import pandas as pd
from datetime import datetime, timedelta

def get_top_retry_dates_after(card_number, card_brand, currency_code, amount, after_date, days_ahead=30, top_k=4):
    model = joblib.load('./enterprise_retry_model.pkl')
    le_card, le_brand, le_currency = joblib.load('./enterprise_encoders.pkl')

    if card_number not in le_card.classes_:
        return ["Unknown card"]

    card_id = le_card.transform([card_number])[0]
    brand_id = le_brand.transform([card_brand])[0]
    currency_id = le_currency.transform([currency_code])[0]

    start_date = pd.to_datetime(after_date).date()
    rows = []
    for i in range(1, days_ahead + 1):
        date = start_date + timedelta(days=i)
        row = {
            'card_number_enc': card_id,
            'card_brand_enc': brand_id,
            'currency_enc': currency_id,
            'transaction_amount': amount,
            'weekday': date.weekday(),
            'month': date.month,
            'is_weekend': int(date.weekday() in [5, 6]),
            'is_month_start': int(date.day <= 3),
            'is_month_end': int(date.day >= 28),
            'txn_index': 500 + i,
            'rolling_success_7d': 0.5,
            'avg_success_rate': 0.5,
            'days_since_last_txn': i
        }
        rows.append((date, row))

    df_future = pd.DataFrame([r[1] for r in rows])
    probs = model.predict_proba(df_future)[:, 1]
    ranked = sorted(zip([r[0] for r in rows], probs), key=lambda x: -x[1])
    return [(d.strftime('%Y-%m-%d'), round(p * 100, 2)) for d, p in ranked[:top_k]]


# === Example usage ===
if __name__ == '__main__':
    sample_input = {
        'card_number': 'card_0',
        'card_brand': 'VISA',
        'currency_code': 'USD',
        'transaction_amount': 120.0,
        'after_date': datetime(2025, 3, 22)
    }

    recommended_dates = get_top_retry_dates_after(
        sample_input['card_number'],
        sample_input['card_brand'],
        sample_input['currency_code'],
        sample_input['transaction_amount'],
        sample_input['after_date']
    )

    print(f"\nTop retry dates after {sample_input['after_date'].date()} with confidence scores:")
    for date, confidence in recommended_dates:
        print(f"{date} - Confidence: {confidence}%")


Top retry dates after 2025-03-22 with confidence scores:
2025-03-23 - Confidence: 67.7699966430664%
2025-04-06 - Confidence: 63.689998626708984%
2025-04-13 - Confidence: 63.43000030517578%
2025-04-05 - Confidence: 62.900001525878906%
